# Conversion of IBM floating points in Python

Recently, I have decided that it is time to make the switch from Python 2.7 to Python 3.5 or 3.6.  My machine currently has 3.5, so it will be that for now.  In addition to fixing all of the minor things that could break my code (like integer division without importing __future__, which actually appeared more frequently than I would have cared for), there are some other code-breaks that I have to fix in my code that I use every day.

I spend a lot of my time working with a file format called a SEG-Y.  SEG-Y is the primary standard format for storing and transmitting seismic field data, processed images, etc.  If you aren't in geophysics, don't worry if you've never heard of it.  If you are, then good for you!  Lets have a beer sometime.

Anyway, when I first began in this job, one of the first things I wrote was a SEG-Y read/write class.  In that I actually subclassed file, so that I could do things like seek to the seismic trace (an array of floats for you non-geophysicists) of my choosing and read a trace and header with readline().  Later I added functionality to it to support iteration and indexing...But that was years ago, and I've been happily using it ever since.  Even though I kind of cringe every time I look at the code.  It's ugly.  There are a number of places that I could do things in a cleaner manner, and it's likely a little less efficient than it could be.  But I do like using it, because it makes a lot of what I do easy.

So this move to Python 3 is the perfect opportunity to refactor and reformulate this code, just because.

And in doing so, I ran into my first problem that I did not expect.

In [1]:
import numpy as np
import scipy.weave as weave

ImportError: No module named 'scipy.weave'

That second line...weave.  It's quite possible that many of you have never ever ever heard of weave.  I kind of think of it as a precursor to numba.  It was one way that was available in 2009 for writing C-ish code that operated on numpy arrays without having to go out and write your own C-extension.  At the time, it was perfect for my needs, because I only ever used it for ONE thing.  Later, when I needed to get to compiled speed, I would use numba...because by that time, just compiling my weave code would give me deprecation warnings.  And now, weave has been completely eliminated.  It is GONE from scipy in Python 3, yielding that territory to numba, cython, ctypes, etc.  (Actually, it was spun off from scipy too, to exist as it's own package, just called weave...but the fact that that import works on my python 2 code only serves to illustrate that I haven't updated scipy in a little while).

So what did I use it for?  What was the big heavy load that it was carrying?  Well, it goes back to SEG-Y files.  You see, before SEG-Y rev 1, (And by the way, this year, the standard for SEG-Y rev 2 was released...so another reason to refactor/update this code...gotta be able to handle those, even though we haven't actually seen any data in that format yet)...anyway, before rev 1 (2002), the original SEG-Y specification, which was published in 1975, allowed data to be stored in a few formats.  Two's-compliment integers of varying size (up to 4 whole bytes!!), 4-byte Fixed point decimals (which rev 1 calls "obsolete", but still supports), and 4 byte floats.

But those 4 byte floats are kind of an issue.  Because the original 1975 SEG-Y specification mandated that they be big-endian IBM floating point numbers.

We don't have any machines in my office, or that I own that can natively use IBM floating point numbers.  Everything we use are little-endian IEEE-754 floats.  So my code that utilizes weave has one very specific job.  Take an array of IBM floating point numbers and convert them to IEEE floating point numbers.  And it does it well.  But rather than just simply re-writing my old as a python function and wrapping it up with numba (easy), I thought it would be fun to go through some of the various ways this could be done.

Plus there are a couple of teeny-tiny little issues with the old code that I have. First and foremost, that algorithm I used in that old code was borrowed from the implementation of the conversion in Seismic Unix.  I had done that primarily because I needed to spin up my code quickly and didn't want to think about the pesky conversion.  I admit that until I sat down to start re-writing the code (and this post), I hadn't thought about how it works at all.  But as I began to examine it and understand it, I wondered if I could do better.

That bit of code I had borrowed is fast, but it is not complete.  First, it completely ignores any idea of utilizing the de-normalized format for floating point magnitudes less than 2<sup>-126</sup>.  Secondly, if somehow you screw things up and don't byteswap the numbers before going into that bit of code, it's likely you will get it stuck in an infinite loop.  Now, even if it didn't you'd get garbage out...but somehow I find that preferable to getting stuck in an infinite loop.

In the scheme of things, those two things are not wildly important.  The byte-swapping and calls to this function are buried in more convienent functions that the user is supposed to use, so they don't have to worry about those pesky infinite loops.  And thus far I haven't really run into any need for using the denormalized floating point values that IEEE can support.  But who knows...maybe that could change someday.  So, for the sake of completeness, I wanted to take some time to really understand all of this, and see if I can't make these functions more complete, and maybe even faster in the process.



## IBM vs IEEE floats (32 bit)

Most of the machines you and I use on a daily basis natively process IEEE-754 floating point numbers.  Usually little-endian as well (endian-ness simply refers to the ordering of the bytes).  Swapping the byte order is trivial when using numpy arrays (simply call byteswap()!), so I won't cover that in here.  The non-trivial part though is converting from IBM to IEEE format.

The IBM format, for 4 byte floats, looks like so:
- A sign bit
- A 7-bit exponent
- A 23 bit significand
- A zero at the end

The IEEE format looks similar at first.  It has:
- A sign bit
- An 8-bit exponent
- A 23 bit significand


But the differences are a bit more than just a differing length exponent.  Where the exponent (e) for IEEE floats specifies multiplication of the significand by 2<sup>(e-127)</sup>  (or 2<sup>(e-126)</sup> for the denormalized floats), the exponent for IBM floats specifies multiplication of the significand by 16<sup>(e-64)</sup>.  Additionally, for normalized IEEE floats (that is floats with an absolute value greater than 2<sup>-126</sup>), The mantissa contains an implied 1 at the beginning.  So all of the 23 bits in the significand represent the stuff that comes after the decimal point.  For IBM floats, there is always an implied zero before the decimal point.  There is no concept of normalized or de-normalized values.

This means that the largest representable number as a single precision IBM float is somewhere around 10<sup>75</sup>, where the largest IEEE float is somewhere around 10<sup>38</sup>.  So our code has to do something for numbers larger than 10<sup>38</sup>.  The larger base for IBM floats also means that they are somewhat less precise than the base-2 IEEE numbers, but we aren't going to be doing any calculation in the IBM format, so we don't really care about that...Just converting, as best we can, to IEEE.  Because of the difference in the bases though, we are likely to lose a little more precision in the conversion.  There's nothing that can be done about that.

Fortunately, rev 1 also allows for IEEE floats to be used, so I don't need to make the conversion go the other way.  I just have to make sure I set the appropriate bits in the header of any SEG-Y files I create.  Easy peasy there.

## Making the conversion

The basic idea is the following:
- 1: Get the sign bit.          (x & 0x80000000)
- 2: Get the base 16 exponent.  (e=(x & 0x7f000000) >> 24)
- 3: Subtract 64.               (e-= 64)
- 4: Multiply the exponent(e) by 4.  (e <<= 2)
- 5: Subtract another 24 from the exponent...because we are going to treat entire mantissa as an integer and it's 24 bits long.
- 6: Get the mantissa (x & 0x00ffffff)
- 7: Now return a float from: f = (-1)<sup>sign</sup> * m * 2<sup>e</sup>



Let's try a dead-simple numpy based function to do this:

 

In [2]:
#First, we have to get some data.  

with open("../SeismicDatasets/test.sgy",'rb') as f1:
    f1.seek(3600+240)  #Go to the first trace data.  Skip the 3200 byte text header, 400 byte binary header, 
                       #and 240 byte trace header fo the first trace
    a = f1.read(1501*4) #Read in the first trace, which I know to have 1501 samples of 4byte IBM floats

In [3]:
import numpy as np

def bytes_to_float_pure_np(big_endian_bytes):
    tr = np.frombuffer(big_endian_bytes, 'i4').byteswap() #Read the bytes object into a numpy array and byteswap
    sign = (tr & 0x80000000) >> 31
    exponent = ((tr &0x7f000000)>>22) - 280  #This is a condensation of steps 2-5
    mantissa = tr & 0x00ffffff
    #return ((-1)**sign)*(mantissa*(2**exponent))
    return ((-1)**sign)*np.ldexp(mantissa, exponent)

z = bytes_to_float_pure_np(a)
%timeit bytes_to_float_pure_np(a)
z

10000 loops, best of 3: 47 µs per loop


array([ -1.99506625e+05,  -7.24510750e+05,  -7.77255500e+05, ...,
         1.51021622e+02,   6.70275000e+03,   5.02452344e+03])

That's not half bad. ~50 microseconds to do all of that for a single trace.  But that's hardly the best we can do.  That function above creates a number of temporary intermediate arrays, and every single time it does it makes another pass through the data.  But its illustrative to do this first because, ideally, that's how simple we'd like it to be.  Interstingly, making the calculation ourselves takes twice as long as using the numpy function ldexp.  Additionally, Note that the dtype here float64 instead of float32, like we might expect.  ldexp, it seems, defaults to double precision. (and strangely, I can't find a way to make it output 32 bit floats at all)  That's not exactly what I wanted...but it does take care of the overflow and underflow issues.  There are other routines I often use as well that default to 64 bit floating point output, regardless of the inputs. So I always have to make sure that I convert back to 32 bit floats before I write out to disk.  So it's not a huge deal really to have the output come at me as 64 bit floats.


Ideally, I'd do one and only one pass through the data.  And I'd like to get back 32 bit floats instead of 64.  Let's try the same thing, but using the numpy "vectorize" routine.  This won't be as fast, because the math operations will be pure-python...But just to outline how this will go down before we wrap it up with numba.


In [4]:
import math
def ibm2ieee_py(x):
    #Convert one IBM float(interpreted as a 4-byte integer) to IEEE float (also interpreted as a 4-byte integer)
    if x:
        x = (((x << 24) & 0xff000000) | ((x >> 24) & 0xff) | #Perform byteswapping!
            ((x << 8) & 0xff0000) | ((x >> 8)) & 0xff00 )  
        mant = 0x00ffffff & x
        exp = int((0x7f000000 & x) >> 22) - 280
        sign = 0x80000000 & x
        x = math.ldexp(mant,exp) 
    return -x if sign else x

ibm2ieee_vectorize = np.vectorize(ibm2ieee_py,['f4'])


def bytes_to_float_py_np(big_endian_bytes):
    tr = np.frombuffer(big_endian_bytes, 'i4')
    return ibm2ieee_vectorize(tr)

zz = bytes_to_float_py_np(a)
%timeit zz=bytes_to_float_py_np(a)

100 loops, best of 3: 2 ms per loop


So we've slowed down the conversion by a factor of 40 or so, but we do only make one pass through the data and create no temporary arrays.  So that's nice.

What if we wrap this up with numba instead of simply using numba's vectorize decorator?

In [5]:
from numba import vectorize, int32, uint32, float32

@vectorize([float32(uint32)])
def ibm2ieee_numba(x):
    if x:
        x = (((x << 24) & 0xff000000) | ((x >> 24) & 0xff) | #Perform byteswapping!
            ((x << 8) & 0xff0000) | ((x >> 8)) & 0xff00 )  
        mant = 0x00ffffff & x
        exp = int((0x7f000000 & x) >> 22) - 280
        sign = 0x80000000 & x
        x = math.ldexp(mant,exp)
    return -x if sign else x

def bytes_to_float_numba_ldexp(big_endian_bytes):
    tr = np.frombuffer(big_endian_bytes,'u4')
    return ibm2ieee_numba(tr)

%timeit y = bytes_to_float_numba_ldexp(a)


10000 loops, best of 3: 22.3 µs per loop


This is a marked improvement.  Twice as fast as the pure numpy example.  Excellent.  I have a hunch though that the call to ldexp is actually slower, especially in the compiled version, than it would be if we computed the result by hand.

In [7]:
@vectorize([float32(uint32)])
def ibm2ieee_numba2(x):
    if x:
        x = (((x << 24) & 0xff000000) | ((x >> 24) & 0xff) | #Perform byteswapping!
            ((x << 8) & 0xff0000) | ((x >> 8)) & 0xff00 )  
        mant = 0x00ffffff & x
        exp = int((0x7f000000 & x) >> 22) - 280
        sign = 0x80000000 & x
        #x = math.ldexp(mant,exp)
        x = mant*(2.0**exp)
    return -x if sign else x

def bytes_to_float_numba_no_ldexp(big_endian_bytes):
    tr = np.frombuffer(big_endian_bytes,'u4')
    return ibm2ieee_numba2(tr)

x = bytes_to_float_numba_no_ldexp(a)
%timeit x = bytes_to_float_numba_no_ldexp(a)
x

100000 loops, best of 3: 16.8 µs per loop


array([ -1.99506625e+05,  -7.24510750e+05,  -7.77255500e+05, ...,
         1.51021622e+02,   6.70275000e+03,   5.02452344e+03], dtype=float32)

That's a small speedup.  But not as much as I'd hoped for.  Incedentially, it's pretty much identical to simply replacing math.ldexp with np.ldexp.  The code I'm trying to replace actually gets about another factor of two speedup relative to this.  But as I mentioned before, that old code is incomplete.  Let's have a look at it.

Below I will replicate it as a numba-vectorized routine, rather than the scipy.weave'd string I used before.  Note that there is no exponentiation in this routine.  We simply create the IEEE floating point representation of the number directly.  But again, we only create normalized IEEE floats.

There are a few main differences to note.  First is the treatment of the exponent.  Like before, we read the 7 bytes after the sign bit and multiply by 4.  But since we are not going to do exponentiation, we want to form the unbiased base two exponent.  So if you will follow me through a tiny bit of math:
- 16<sup>(x-64)</sup> = 2<sup>(y-127)</sup>
- 2<sup>(4*x-4*64)</sup> = 2<sup>(y-127)</sup>
- Now we take the base two log of both sides
- 4*x - 256 = y-127
- y = 4*x - 129

So our base two unbiased exponent is 4 times the base 16 unbiased exponent minus 129.  BUT, we are forgetting one thing.  In the IBM representation, the mantissa has an implicit 0 before the decimal point, whereas the IEEE representation has an implicit 1 before the decimal point (when using the normalized formulation).  So, to move the decimal point, we will continuously multiply the IBM mantissa by two (bit-shifting to the left) until we have a one in the 25th byte location.  Each time we do that, we reduce the value of our base two exponent by one to represent the same number.  But we can take care of the first multiplication at the outset, which means we subtract 130 instead of 129, and do our multiplication loop until we get a one in the 24th bit.  This means if there is a one already there, we don't have to do any more bit-shifting and exponent reducing.  Then at the end, we simply stick all the bits together.  The sign bit, the new unbiased exponent, and the last 23 bits of the mantissa, lopping off the implied one that we have moved to the 24th bit location.  Boom, done.  Well, almost...we have to handle the cases of the exponent being too large or too small as well.

One other thing to note:  this function goes uint->uint.  So after we are done, we have to tell the numpy array to interpret it's bytes as a 4-byte float.

In [8]:
@vectorize([uint32(uint32)])
def ibm2ieee_numba3(x):
    if x:
        x = (((x << 24) & 0xff000000) | ((x >> 24) & 0xff) | #Perform byteswapping!
            ((x << 8) & 0xff0000) | ((x >> 8)) & 0xff00 )  
        mant = 0x00ffffff & x                
        e = int( (0x7f000000 & x) >> 22 ) - 130  
        while not (mant & 0x00800000):        #Keep moving decimal point right until a 1 is in the first place
            mant <<= 1
            e -= 1
        if e> 254:
            x = (0x80000000 & x) | 0x7f7fffff #Set to largest possible float (clip)
        elif e < 0:
            x = 0
        else:
            x = (0x80000000 & x) | (e << 23) | (0x007fffff & mant)
    return x

def bytes_to_floats_numba_direct(big_endian_bytes):
    tr = np.frombuffer(big_endian_bytes, dtype='u4')
    converted_array = ibm2ieee_numba3(tr)
    converted_array.dtype='f4'
    return converted_array

xx=bytes_to_floats_numba_direct(a)
%timeit xx=bytes_to_floats_numba_direct(a)
xx

100000 loops, best of 3: 7.14 µs per loop


array([ -1.99506625e+05,  -7.24510750e+05,  -7.77255500e+05, ...,
         1.51021622e+02,   6.70275000e+03,   5.02452344e+03], dtype=float32)

There's our factor of two speedup.  And we can see that we are getting the same results in our array.  In this case we have not at all handled any underflow, though I don't think any is occuring in this particular array, because as we can see, it is equal to the array produced by our first, numpy only method.


I can speed it up a tiny bit more by placing the most often called condition (the one where we actually make the number) first

In [9]:
@vectorize([uint32(uint32)])
def ibm2ieee_numba4(x):
    if x:
        x = (((x << 24) & 0xff000000) | ((x >> 24) & 0xff) | #Perform byteswapping!
            ((x << 8) & 0xff0000) | ((x >> 8)) & 0xff00 )  
        mant = 0x00ffffff & x                
        e = int( (0x7f000000 & x) >> 22 ) - 130
        while not (mant & 0x00800000):        #Keep moving decimal point right until a 1 is in the first place
            mant <<= 1
            e -= 1
        if 0 <= e < 255:
            x = (0x80000000 & x) | (e << 23) | (0x007fffff & mant)
        elif e> 254:
            x = (0x80000000 & x) | 0x7f7fffff #Set to largest possible float (clip)
        else:
            x = 0
            
    return x

def bytes_to_floats_numba_direct2(big_endian_bytes):
    tr = np.frombuffer(big_endian_bytes, dtype='u4')
    converted_array = ibm2ieee_numba4(tr)
    converted_array.dtype='f4'
    return converted_array

xx=bytes_to_floats_numba_direct2(a)
%timeit xx=bytes_to_floats_numba_direct2(a)
np.array_equal(xx,z)

100000 loops, best of 3: 6.75 µs per loop


True

I think that's about as fast as I'm going to get this...But now I want to make some improvements.  The first is easy:  remember when I said that it's possible to get into an infinite loop in that while statement if you haven't byte-swapped first?  Let's protect against that. So let's add a condition to the while loop that we only loop if e>0.  That's going to slow things down a teeny-tiny bit for the checking of e.

Additionally, let's put in some underflow handling, shall we?  IEEE-754 allows for handling of floats less than 2<sup>-126</sup> through it's de-normalized format, in which there is no leading one implied before the radix point.  We can actually handle numbers down to 2<sup>-126-23</sup> = 2<sup>-149</sup>.  In those cases, we simply set the exponent to zero and let the mantissa go as small as we like.  Actually, thinking about this has led me to notice something...do you notice that we let e go all the way to zero before we set x=0 before?  In any cases where we have done this, we've actually told IEEE to use the denormalized version...but we have left the mantissa as a number that implies a 1 before the radix point.  So previously, we've actually mis-handled numbers where e=0.  Let's fix that too.  

So if e does get to zero...then just use the first 23 bits of the mantissa instead of the last 23.  Further, if e starts off less than zero, shift the mantissa -e+1 bits to the right to capture whatever we can.  In fact, by doing this, we can eliminate the else that sets x = 0, and pick up signed zeros for the underflow.  Even though IBM floats do not specify a signed zero, we can.

In [10]:
@vectorize([uint32(uint32)])
def ibm2ieee_numba5(x):
    if x:
        x = (((x << 24) & 0xff000000) | ((x >> 24) & 0xff) | #Perform byteswapping!
            ((x << 8) & 0xff0000) | ((x >> 8)) & 0xff00 )  
        mant = 0x00ffffff & x                
        e = int( (0x7f000000 & x) >> 22 ) - 130
        while not (mant & 0x00800000) and e>0:  #Keep moving decimal point right until a 1 is in the first place
            mant <<= 1
            e -= 1
        if 0 < e < 255:
            x = (0x80000000 & x) | (e << 23) | (0x007fffff & mant)
        elif e > 254:
            x = (0x80000000 & x) | 0x7f7fffff #Set to largest possible float (clip)
        #elif e > -23:
        else:
            x = (0x80000000 & x) | (0x007fffff & (mant >> (-e+1))) #handle underflow up to capabilities of IEEE
            
    return x

def bytes_to_floats_numba_direct3(big_endian_bytes):
    tr = np.frombuffer(big_endian_bytes, dtype='u4')
    converted_array = ibm2ieee_numba5(tr)
    converted_array.dtype='f4'
    return converted_array

xx=bytes_to_floats_numba_direct3(a)
%timeit xx=bytes_to_floats_numba_direct3(a)
np.array_equal(xx,z)

100000 loops, best of 3: 7.9 µs per loop


True

In [11]:
#For testing, we are just going to use a python version of the routine, and we won't be doing any byteswapping!

def ibm2ieee_py_final(x):
    if x:
        #x = (((x << 24) & 0xff000000) | ((x >> 24) & 0xff) | #Perform byteswapping!
        #    ((x << 8) & 0xff0000) | ((x >> 8)) & 0xff00 )  
        mant = 0x00ffffff & x                
        e = int( (0x7f000000 & x) >> 22 ) - 130
        while (not (mant & 0x00800000)) and e>0:  #Keep moving decimal point to left until a 1 is in the first place
            mant <<= 1
            e -= 1
        if 0 < e < 255:
            x = (0x80000000 & x) | (e << 23) | (0x007fffff & mant)
        elif e > 254:
            x = (0x80000000 & x) | 0x7f7fffff #Set to largest possible float (clip)
        #elif e > -23:
        else:
            x = (0x80000000 & x) | (0x007fffff & (mant >> (-e+1))) #handle underflow up to capabilities of IEEE
        #else:
        #    x = 0
    return x


Now I need some tests to test the underflow handling and a few edge cases.  Let's start with the middle denormalized IEEE float.  It will looks like this:
- exponent = 0000 0000
- mantissa = 100 0000 0000 0000 0000

which means 2<sup>-1</sup> * 2<sup>-126</sup> = 2<sup>-127</sup>

As a uint, it looks like this:

In [12]:
middle_ieee = 0b00000000010000000000000000000000
middle_ieee

4194304

As an IBM float though, that would be:
2<sup>-127</sup> = 16<sup>-31</sup> * 2<sup>-3</sup>

- So our exponent would be -31+64 = 33, =  0100001
- and our mantissa would be 0010 000 ... 0000

Or as a uint:

In [13]:
ibm_middle_ieee = int(0b00100001001000000000000000000000)
ibm_middle_ieee


555745280

In [14]:
ibm2ieee_py_final(ibm_middle_ieee)

4194304

In [15]:
ibm2ieee_py_final(ibm_middle_ieee) == middle_ieee

True

Great...so it handles perfectly the case where we work our way down to zero from a larger than zero exponent.  So far so good.  What about the case where we have an exponent that is less than zero to start with?.  What would the smallest denormalized IEEE float look like?

That's simply a 1 in the very last bit.  It represents 2<sup>-126</sup> * 2<sup>-23</sup> = 2<sup>-149</sup>

As an IBM float, that's 16<sup>-37</sup> * 2<sup>-1</sup>, so it will have an exponent of 27 (0011011) and a mantissa of 1000 0000 .... 0000

In [16]:
smallest_ieee = 0b00000000000000000000000000000001
smallest_ieee

1

In [17]:
ibm_smallest_ieee = 0b00011011100000000000000000000000
ibm_smallest_ieee

461373440

In [18]:
ibm2ieee_py_final(ibm_smallest_ieee)

1

In [19]:
ibm2ieee_py_final(ibm_smallest_ieee) == smallest_ieee

True

So, to wrap things up, I have re-implemented single precision IBM float conversion as a numba vectorized function, because that seems to be the fastest I can get with ease.  I don't feel incredibly comfortable with Cython at the moment to see if I could perhaps eek out more speed by doing the same thing with it, but my feeling is that it probably wouldn't be significantly faster, if it's faster at all.  In addition to re-implementing the method I had used before, I also made it more robust (impossible to get stuck in an infinite loop, no matter how mangled the data are.  Yay!), and more complete than it had been before by adding the capability of handling underflow with de-normalized IEEE floats.

So, just to re-package it all at the end, here is the routine I will be using in it's final form.  I hope you have enjoyed this little voyage through a data-type from 1975.



In [20]:
@vectorize([uint32(uint32)])
def ibm2ieee(x):
    if x:
        x = (((x << 24) & 0xff000000) | ((x >> 24) & 0xff) | #Perform byteswapping!
            ((x << 8) & 0xff0000) | ((x >> 8)) & 0xff00 )  
        mant = 0x00ffffff & x                
        e = int( (0x7f000000 & x) >> 22 ) - 130
        while (not (mant & 0x00800000)) and e>0:  #Keep moving decimal point right until a 1 is in the first place
            mant <<= 1
            e -= 1
        if 0 < e < 255:
            x = (0x80000000 & x) | (e << 23) | (0x007fffff & mant)
        elif e > 254:
            x = (0x80000000 & x) | 0x7f7fffff #Set to largest possible float (clip)
        else:
            x = (0x80000000 & x) | (0x007fffff & (mant >> (-e+1))) #handle underflow up to capabilities of IEEE
            
    return x

def bytes_to_floats(big_endian_bytes):
    tr = np.frombuffer(big_endian_bytes, dtype='u4')
    converted_array = ibm2ieee(tr)
    converted_array.dtype='f4'
    return converted_array

xx=bytes_to_floats(a)
%timeit xx=bytes_to_floats(a)
np.array_equal(xx,z)

100000 loops, best of 3: 7.93 µs per loop


True

I will likely be revisiting this in the future.  I just started reading the rev2 standard, and among other things, it supports little-endian encoding, so I'll be modifying the above code to make the byteswapping optional. (or using two versions, one with and one without...that's probably better...no unnecessary if statements).  Fortunately, I will not have to write a version to handle double precision IBM floats.  Though I suppose that's probably wrong.  It's not uncommon to get files that don't completely comply with the standard...I imagine I'll see one in 64-bit IBM format sometime next year.  Woo-hoo.